In [1]:
# MY SIMULATION 
import itertools
%matplotlib inline
import os
import random 
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML
import vertex_model as model
# from vertex_model.run_select_nosaveinfo import run_simulation_INM, definecolors, run_simulation_no_INM
import itertools
import numpy as np
import matplotlib.pyplot as plt
import vertex_model as model
import vertex_model.initialisation as init
import vertex_model.characterization as crt
from vertex_model.forces import TargetArea, Tension, Perimeter, Pressure
import os
import seaborn as sns
import warnings
import matplotlib.colors as colors
warnings.filterwarnings('ignore') #Don't show warnings
#from Gobal_Constant import dt, viscosity, t_G1, t_G2, t_S, A_c, J, pos_d, T1_eps, P, microns, time_hours, expansion_constant #file with necessary constants
from vertex_model.run_select import save_data
sns.set_style("whitegrid")
from scipy.spatial import Delaunay
import math
from scipy.spatial import ConvexHull

In [11]:
#Global constants
#Global constants
dt=0.001            #time step
viscosity= 0.02  #viscosity*dv/dt = F
P= 0.0
expansion_constant = 1
Number_simulations = 50
K=1.0 #area elasticity
G= 0.04 #contractility of the cell
L=0.075 #line tensions
Lambda_0 = 0.68 #Line tensions can be reduced by increasing cell-cell adhesion or reducing actin- myosin contractility
t_end = 30
area_dimension= 177 #np.pi*(15/2.0)**2 #15 micrometros de diámetro

In [3]:
# run simulation
def run(simulation,N_step,skip):
    N_step = math.floor(N_step)
    skip = math.floor(skip)
    iter =  itertools.islice(simulation,0,N_step,skip)
    return [cells.copy() for cells in itertools.islice(simulation,0,N_step,skip)]

#simulation without division
def basic_simulation(cells,force,dt=dt,T1_eps=0.04):
    expansion = np.array([0.0,0.0])
    while True:
#         print cells.properties['parent_group']
        # cells.mesh , number_T1, edg_rem = cells.mesh.transition(T1_eps)
#         cells.properties['Gamma'][cells.mesh.face_ids == 50]=np.array([0.0])
        F = force(cells)/viscosity
#         expansion = 0.05*np.average(F*cells.mesh.vertices,1)*dt
        dv = dt*model.sum_vertices(cells.mesh.edges,F) 
#         cells.mesh = cells.mesh.moved(dv).scaled(1.0+ expansion)
        if hasattr(cells.mesh.geometry,'width'):
            expansion[0] = expansion_constant*np.average(F[0]*cells.mesh.vertices[0])*dt/(cells.mesh.geometry.width**2)
        if hasattr(cells.mesh.geometry,'height'): #Cylinder mesh doesn't have 'height' argument
            expansion[1] = np.average(F[1]*cells.mesh.vertices[1])*dt/(cells.mesh.geometry.height**2)
        cells.mesh = cells.mesh.moved(dv).scaled(1.0+expansion)
        yield cells

In [4]:
def definecolors(cells):
    peach = '#eed5b7'
    light_blue ='#87cefa'
    pink = '#ffc0cb'
    light_green = '#98fb98'
    import matplotlib.colors as colors
    vv=sns.color_palette("hls", 10)
    v=[colors.rgb2hex(colorrgb) for colorrgb in vv]
    palette = np.array([light_green, pink,light_green,'g','r','g','m','c','',peach])
    palette = np.array([v[0],v[1],v[1], v[1],v[4],v[5],v[6],v[7],v[8],v[9],peach])
    colors = cells.properties['parent_group']
    return palette[colors]

In [5]:
## Extrusion Time 
def t_extrusion(simulation, id_ecad_cells): 
    t = 0
    for time_step in simulation:
        extrused_cells = 0
        t = t+1
        area = time_step.mesh.area
        for a in area:
            if a == 0:
                extrused_cells= extrused_cells+1
        if extrused_cells == len(id_ecad_cells): 
            break
    return t

In [6]:
def cell_vertices(mesh, id):    
    cell_i = []
    for i in range(len(mesh.face_id_by_edge)):
        if mesh.face_id_by_edge[i] == id:
            cell_i.append(i)
    return cell_i
def centroid(vertexes_x, vertexes_y):   
     _len = len(vertexes_x)
     _x = sum(vertexes_x) / _len
     _y = sum(vertexes_y) / _len
     return(_x, _y)
def mesh_centres(mesh):
    '''
    Compute the centres of the cells of a given mesh 
    '''
    centres_x =[]
    centres_y = [] 
    centres =np.zeros((mesh.n_face, 2))
    for i in range(mesh.n_face):       
       cell_i = cell_vertices(mesh, i) #vertices of the ith cell 
       if len(mesh.vertices[0][cell_i]) != 0:
        cell_center_i = centroid(mesh.vertices[0][cell_i], mesh.vertices[1][cell_i])
        centres_x.append(cell_center_i[0])
        centres_y.append(cell_center_i[1])
        #centres.append(np.array([cell_center_i[0],cell_center_i[1] ]))
        centres[i, 0] = cell_center_i[0]
        centres[i, 1] = cell_center_i[1]
    return centres_x, centres_y, centres
def circumcenter(vertices):
    hull = ConvexHull(vertices)
    edges = hull.points[hull.simplices]

    circumcenters = []
    for edge in edges:
        x1, y1 = edge[0]
        x2, y2 = edge[1]
        x3, y3 = edge[2]

        A = np.array([[x2 - x1, y2 - y1], [x3 - x1, y3 - y1]])
        b = 0.5 * np.array([[(x2 - x1) ** 2 + (y2 - y1) ** 2], [(x3 - x1) ** 2 + (y3 - y1) ** 2]])
        try:
            circumcenter = np.linalg.solve(A, b)
            circumcenter = [circumcenter[0][0] + x1, circumcenter[1][0] + y1]
            circumcenters.append(circumcenter)
        except np.linalg.LinAlgError:
            pass

    circumcenter = np.mean(circumcenters, axis=0)
    return circumcenter



In [8]:
def save_evolution2(simulation, path, id_ecad_cells, s): #sim is a list of nsim sims same noise, same mc 
    area_all = []; length_distortion_all = []; angle_distortion_all = [];edges_length_all = []
    si_all =[]; te_all = []
    for sim in simulation: 
        area = []; length_distortion = []; angle_distortion = [];edges_length = []
        si =[]; te = []
        # we have to go through all time steps of the simulation 
        n_steps = len(sim)
        i  = 0
        t= t_extrusion(sim, id_ecad_cells)
        te.append(t)
        while i < n_steps:
            print(i)
            mesh_i = sim[i].mesh
            centres_x, centres_y, centres = mesh_centres(mesh_i)
            tri = Delaunay(centres)
            triangles = centres[tri.simplices]  
            non_obtuse, non_obtuse_indices, obtuse, obtuse_indices = crt.triangle_sieve(triangles, np.pi / 10, np.pi - (np.pi / 10))
            # trisieve = tri.simplices[non_obtuse_indices]
            #plt.triplot(centres[:,0], centres[:,1], trisieve)
            length_distortion.append(np.mean(crt.length_distortion(non_obtuse)))
            angle_distortion.append(np.mean(crt.angle_distortion(non_obtuse)))
            edges_length.append(crt.mean_perimeter(non_obtuse))
            a, std = crt.mean_area_triangles(non_obtuse)
            area.append(a)
            shape_index  = crt.shape_index(mesh_i); si.append(shape_index)
            i  =i +s
        area_all.append(area); length_distortion_all.append(length_distortion)
        angle_distortion_all.append(angle_distortion)
        si_all.append(si); edges_length_all.append(edges_length);
    # once the loop over all sims is over: 
    mean_areas = np.mean(area_all,0); mean_edges = np.mean(edges_length_all,0); 
    mean_ld= np.mean(length_distortion_all,0); mean_ad = np.mean(angle_distortion_all,0); 
    mean_t= np.mean(te_all); mean_si = np.mean(si_all,0); 

    std_areas = np.std(area_all,0); std_edges = np.std(edges_length_all,0); 
    std_ld= np.std(length_distortion_all,0); std_ad = np.std(angle_distortion_all,0); 
    std_t= np.std(te_all); std_si = np.std(si_all,0); 

    # save mean
    file_ld = os.path.join(path,  'length_distortion.txt')
    np.savetxt(file_ld, mean_ld, delimiter=',')

    file_ad = os.path.join(path, 'angle_distortion.txt')
    np.savetxt(file_ad, mean_ad, delimiter=',')

    file_el = os.path.join(path, 'edges_length.txt')
    np.savetxt(file_el, mean_edges, delimiter=',')

    file_a = os.path.join(path, 'area.txt')
    np.savetxt(file_a, mean_areas, delimiter=',')

    file_si = os.path.join(path, 'shape_index.txt')
    np.savetxt(file_si, mean_si, delimiter=',')

    # save standard deviation
    file_ld = os.path.join(path,  'length_distortion_std.txt')
    np.savetxt(file_ld, std_ld, delimiter=',')

    file_ad = os.path.join(path, 'angle_distortion_std.txt')
    np.savetxt(file_ad, std_ad, delimiter=',')

    file_el = os.path.join(path, 'edges_length_std.txt')
    np.savetxt(file_el, std_edges, delimiter=',')

    file_a = os.path.join(path, 'area_std.txt')
    np.savetxt(file_a, std_areas, delimiter=',')

    file_si = os.path.join(path, 'shape_index_std.txt')
    np.savetxt(file_si, std_si, delimiter=',')

    file_te = os.path.join(path, 'time_extrusion.txt')
    np.savetxt(file_te, te, delimiter=',')
    

In [9]:
# to save data from simulation
def save_simulation2(I, data, mutant_cells, id_ecad_cells, date, size, noise, s): # data is going to be the simulation (only one) 
    """
    I : number of the simulation
    data: simulation data
    name_folder: name of the folder where data is kept. It has to include the percentage of mutant
                cells of the current simulation 
    """
    # main folder 
    path=r'C:\Users\natal\OneDrive\Documentos\MASTER\TFM\Simulations' 
    name_folder = 'sim_time_'+'%0.3f'%mutant_cells + f'_{size}'+f'_{noise}_' + date

    new_path = os.path.join(path, name_folder)
    if not os.path.exists(new_path):
        os.makedirs(new_path)
    # last meshes folder 
    
        
    save_evolution2(data, new_path, id_ecad_cells, s)    

In [14]:
s = 20

In [15]:
simulations_diff_mutant = [] # to store the different simulations for the different values of noise
noise =0
ncellup =18
ncellacross = 18
ncells = ncellacross*ncellup
# ecad_cells_proportion = 0.04
# ecad_cells = int(ecad_cells_proportion* ncells)
# id_ecad_cells = random.sample(range(ncells), ecad_cells); print(id_ecad_cells)
mutant_cells_vec =[0.15]#different proportions of mutant cells in the mesh
ecad_cells_vec = []
for m in mutant_cells_vec:
    print(m)
    history_10={}
    ecad_cells = int(m * ncells)
    id_ecad_cells = random.sample(range(ncells), ecad_cells); 
    ecad_cells_vec.append(id_ecad_cells)
    #ids_Gamma0 = random.sample(range(1, 100), 10)
    simulations_same_mutant = [] # to do the average between 10simulations for each level of mutant cells 
    for i in range(1):
        print(i)
        #run simulation with the choosen parameters
        rand =  np.random.RandomState() #random number to choose Lambda
        params = [K,G,L]  # K=x[0],G=x[1],L=x[2]
        hex_centres = init.hexagonal_centres(ncellup, ncellacross, noise, rand= np.random.RandomState())
        # create a voronoi mesh with those centres 
        #  
        mesh = init.toroidal_ic_mesh(hex_centres)
        cells = model.Cells(mesh,properties={'K':K,'Gamma':G,'P':0.0,'boundary_P':P,'Lambda':L, 'Lambda_boundary':0.5, 'A0':1.0})
        force = TargetArea() + Tension() + Perimeter() + Pressure()
        step = int(50/dt)
        history_init_10= run(basic_simulation(cells,force),step,int(1/dt)) 
        expansion_constant = 1
        cells = history_init_10[-1].copy()
        cells.properties['parent_group'] = np.zeros(len(cells),dtype=int)
        if id_ecad_cells != []:
            cells.properties['parent_group'][np.where([cells.mesh.face_ids == index for index in id_ecad_cells])[1]] = 1
        cells.properties['Gamma']= np.array([G,0])[cells.properties['parent_group']]
        cells.properties['Lambda']= np.array([L,Lambda_0])[cells.properties['parent_group']]
        force = TargetArea() + Tension() + Perimeter() + Pressure()
        N_Step = math.floor(t_end/dt)
        history_10[i]= run(basic_simulation(cells,force),N_Step,.010/dt) 
        #save_data(i,history_10[i],'cells10')
        #save_simulation(i, history_10[i], m,id_ecad_cells,"30aug", ncellacross, noise,s)
        simulations_same_mutant.append(history_10[i])
        print(f'{m} mutant cells, sim number {i}')
    save_simulation2(0,simulations_same_mutant, m,id_ecad_cells,"4sep", ncellacross, noise,s)
    simulations_diff_mutant.append(simulations_same_mutant)


0.15
0
0.15 mutant cells, sim number 0
0
20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480
500
520
540
560
580
600
620
640
660
680
700
720
740
760
780
800
820
840
860
880
900
920
940
960
980
1000
1020
1040
1060
1080
1100
1120
1140
1160
1180
1200
1220
1240
1260
1280
1300
1320
1340
1360
1380
1400
1420
1440
1460
1480
1500
1520
1540
1560
1580
1600
1620
1640
1660
1680
1700
1720
1740
1760
1780
1800
1820
1840
1860
1880
1900
1920
1940
1960
1980
2000
2020
2040
2060
2080
2100
2120
2140
2160
2180
2200
2220
2240
2260
2280
2300
2320
2340
2360
2380
2400
2420
2440
2460
2480
2500
2520
2540
2560
2580
2600
2620
2640
2660
2680
2700
2720
2740
2760
2780
2800
2820
2840
2860
2880
2900
2920
2940
2960
2980


In [ ]:
n_steps = 0.01/dt
time_vector =  np.arange(0, n_steps, s)
